In [1]:
from multicamera_acquisition.interfaces.arduino import (
    packIntAsLong,
    wait_for_serial_confirmation,
)
import struct
import time
import numpy as np

from multicamera_acquisition.interfaces import get_camera
from multicamera_acquisition.video_io import write_frame
from multicamera_acquisition.paths import ensure_dir
from multicamera_acquisition.acquisition import acquire_video

from pypylon import pylon

/home/dattalab1/Desktop/azure_and_basler/multicamera_acquisition/multicamera_acquisition/interfaces/arduino.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
def packIntAsLong(value):
    """Packs a python 4 byte integer to an arduino long"""
    return struct.pack('i', value)

    
def get_timing_params(duration, framerate, exposure_time, buffer=50):
    
    inv_framerate = int(1e6/framerate)
    num_cycles = int(duration * framerate / 2)
    
    params = (
        num_cycles,
        exposure_time,
        inv_framerate,
    )
    
    return params
    


In [3]:
! df -h .

Filesystem      Size  Used Avail Use% Mounted on
/dev/sda1       916G  127G  743G  15% /


## Set up the camera

In [4]:
# cam = get_camera(brand='pylon', exposure_time=1000, readout_mode='Normal')


recording_duration_s = 60*20
save_location = '/home/dattalab1/Desktop/DOI/3_09/'

camera_list = [
    {'name': 'top',    'serial': 24600953, 'brand':'basler', 'gain': 10, 'exposure_time':1100, "display":False},
]

In [5]:
# import subprocess
# subprocess.run('rm -r /home/dattalab1/Desktop/azure_and_basler/notebooks/data/test',shell=True)
# subprocess.run('rm -r /home/dattalab1/Desktop/azure_and_basler/notebooks/data/Untitled',shell=True)

In [ ]:

acquire_video(
    save_location,
    camera_list,
    azure_framerate = 30,  # azure's frame rate in this case.
    basler_framerate=120,
    recording_duration_s = recording_duration_s,
    serial_timeout_duration_s=0.1,
    overwrite=True,
    append_datetime=False,
    verbose=True
)

Using serial port /dev/ttyACM2
36000


/home/dattalab1/Desktop/azure_and_basler/multicamera_acquisition/multicamera_acquisition/interfaces/arduino.py:34: UserWarning: PySerial: "Start" confirmation expected, got "Waiting..."". Trying again.
  warnings.warn(
/home/dattalab1/Desktop/azure_and_basler/multicamera_acquisition/multicamera_acquisition/interfaces/arduino.py:34: UserWarning: PySerial: "Start" confirmation expected, got "0"". Trying again.
  warnings.warn(


Confirmation recieved: Start


recording progress (s):   0%|                          | 0/1200 [00:00<?, ?it/s]

Num cycles:36000
670,8545,16528,24861,


recording progress (s):  98%|██████████████▊| 1181/1200 [19:53<00:37,  1.97s/it]

Finished
End Acquisition


recording progress (s):  98%|██████████████▊| 1181/1200 [20:07<00:19,  1.02s/it]


## DEBUG: simple test of communication with arduino

In [ ]:
import serial
x = 5000
with serial.Serial(port='/dev/ttyACM1', baudrate=9600) as ser:
    if ser.isOpen():

        # Read data from the serial device
        line = ser.readline()  # Read a line of data
        print(f"Received line: {line.decode('utf-8')}")
        
        ser.write(x.to_bytes(4, 'little'))
        
        wait_for_serial_confirmation(
            ser, 
            'Start', 
            recording_duration=5, 
            timeout_duration_s=0.1
        )
        
    else:
        print("Failed to open serial connection.")

In [ ]:
with serial.Serial(port='/dev/ttyACM1', baudrate=9600) as ser:
    if ser.isOpen():
        
        ser.write(x.to_bytes(4, 'big'))
        wait_for_serial_confirmation(
            ser, 
            'Breaking!', 
            recording_duration=5, 
            timeout_duration_s=0.1
        )